# Introduction
In this notebook, we will load an example time series, fit a growth model
and plot the signals.

## Load example time series and config

Let's start by loading example time series data.

In [ ]:
%matplotlib inline
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import staticchar as ch

dataset = ch.Dataset("../tests/test_data/S-shape")
#dataset = ch.Dataset("../staticchar/tests/test_data/SignificantDeath")

Config

In [ ]:
yaml_path = pathlib.Path('../tests/configs/integral_basic.yml')
config = ch.config.load(yaml_path, ch.config.CharacterizationConfig)
config

In [ ]:
raw_timeseries = dataset.get_a_frame()
raw_timeseries.head()

As we can see, there is some non-zero signal at the beginning, which we attribute to
the media absorbance and media fluorescence (as initially we have very low cell density).

In [ ]:
colors = config.colors()
ax = plt.subplot()
ch.plot_signals_against_time(raw_timeseries, signals=colors.keys(), time_column="time", ax=ax, colors=colors)
ax.legend()
sns.despine()

## Pre-processing
Let's assume this is the background and subtract it.
(A more precise, but also costly alternative is to estimate this using several blanks).

In [ ]:
subtracted = ch.subtract_background(raw_timeseries,
                                    columns=config.background_subtract_columns(),
                                    strategy=ch.BackgroundChoices.Minimum)
ax = plt.subplot()
ch.plot_signals_against_time(subtracted, signals=colors.keys(), time_column="time", ax=ax, colors=colors)
ax.legend()
sns.despine()

### Fitting a growth model

Let's fit a growth model to the OD signal.

In [ ]:
model_params = ch.LogisticModel.fit(subtracted["time"], subtracted[config.growth_signal])
model = ch.LogisticModel(model_params)

# model_params = ch.GompertzModel.fit(subtracted["time"], subtracted[config.growth_signal])
# model = ch.GompertzModel(model_params)

print(f"Inferred parameters: {model_params}")
print(f"Growth phase: {model.growth_period}")
print(f"Time of maximal activity: {model.time_maximal_activity}")
print(f"Inferred (log of) initial density: {model.initial_density(log=True)}")

ch.plot_growth_model(subtracted["time"], subtracted[config.growth_signal], model=model)
sns.despine()

### Plotting the data

Some time after the growth phase, we should observe a similar exponential production
of the proteins. Suppose that this maturation time is about 50 minutes,
that is about 0.85 hours.

Then, fluorescence signals should be linear when drawn with respect to each other.

In [ ]:
# Add offset to the growth phase
production_phase = model.growth_period + config.maturation_offset

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ch.plot_signals_against_time(subtracted, signals=colors.keys(), time_column="time", ax=ax1, colors=colors)

# Visualise the production phase
ch.mark_phase(ax1, interval=production_phase, color="green", alpha=0.1)

ch.plot_signals_against_reference(subtracted, signals=("EYFP", "ECFP"), reference="mRFP1", colors=colors, ax=ax2)

sns.despine()

### Truncate the time-series

We see that this very well captures the growth phase of mRFP1 (the reference signal),
but is a bit too late for EYFP and ECFP -- we won't have a linear dependence between
the signals...

Let's choose a more narrow interval.

In [ ]:
another_production_phase = ch.TimePeriod(reference=12, left=2, right=2)
truncated_timeseries = ch.select_time_interval(subtracted, interval=another_production_phase)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ch.plot_signals_against_time(subtracted, signals=colors.keys(), time_column="time", ax=ax1, colors=colors)

# Visualise the production phase
ch.mark_phase(ax1, interval=another_production_phase, color="green", alpha=0.1)

ch.plot_signals_against_reference(truncated_timeseries, signals=("EYFP", "ECFP"), reference="mRFP1", colors=colors, ax=ax2)
sns.despine()

Run method

In [ ]:
gradient_result = ch.transcriptional_activity_ratio(truncated_timeseries, 
    config.signals, 
    config.reference, 
    config.signal_properties,
    model_params.growth_rate, 
    model.growth_period, 
    maturation_offset = config.maturation_offset
    )    
gradient_result

### Integration-based characterization
Now assume that we want to integrate the signals over the production period.

In [ ]:
ch.integrate(data=subtracted, signals=config.signals, interval=config.time_window)

Now plot the output

In [ ]:
f, axs = plt.subplots(1, len(config.signals), figsize=(12,4))
for signal, ax in zip(config.signals, axs):
    ch.plot_integration(subtracted, signal, config.time_window, ax, fillcolor=colors[signal])
    
sns.despine()

## Characterize a whole plate
Using the built-in helper methods, you can characterize a whole plate in a single call

In [ ]:
plate = ch.plate.Plate(dataset, config)
plate.characterize()
plate.to_dataframe()

### Use reference wells to specify time windows

In [ ]:
config.reference_wells = ['064d02dd-f912-49a0-964a-5c758e92367b']
plate = ch.plate.Plate(dataset, config)
plate.characterize()
plate.to_dataframe()

In [ ]:
time_window = config.time_window
f, axs = plt.subplots(len(config.signals), len(dataset), sharex=True, figsize=(10,5))
for signal, axr in zip(config.signals, axs):
    for i, (key, frame) in enumerate(dataset.items()):
        subtracted = ch.preprocessing.subtract_background(frame, columns=config.background_subtract_columns(), strategy=ch.preprocessing.BackgroundChoices.Minimum)
        time_window.reference = plate.reference_time
        ch.plot_integration(subtracted, signal, time_window, axr[i], fillcolor=colors[signal])
plt.tight_layout()
sns.despine()